In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
sys.path.append("../..")
import tensortree
from Bio import SeqIO
import numpy as np
import tensorflow as tf

2024-12-05 10:03:52.466038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 10:03:52.482314: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 10:03:52.487349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 10:03:52.500037: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 10:03:53.173980: W tensorflow/compiler/tf2

In [2]:
msa_file = "seq-gen.out"
tree_file = "RAxML_bestTree.out2"
alphabet = "ACGT"

In [3]:
tensortree.set_backend("tensorflow")

In [4]:
# parse fasta
leaf_names = []
seqs = []
for record in SeqIO.parse(msa_file, "fasta"):
    leaf_names.append(record.id)
    seqs.append(str(record.seq))

one_hot_leaves = tensortree.util.encode_one_hot(seqs, alphabet=alphabet)
one_hot_leaves = one_hot_leaves[:, np.newaxis] # add model dimension

# create tree
tree = tensortree.TreeHandler.read(tree_file)

In [5]:
def inverse_softplus(x):
    return np.log(np.exp(x) - 1 + 1e-16)

# create variables, trivially initialize
R, pi = tensortree.substitution_models.jukes_cantor(d = len(alphabet))
R_kernel = tf.Variable(inverse_softplus(R))
pi_kernel = tf.Variable(np.log(pi))
B_kernel = tf.Variable(inverse_softplus(np.ones_like(tree.branch_lengths)))


def make_R_pi_B(R_kernel, pi_kernel, B_kernel):
    R = tensortree.model.backend.make_symmetric_pos_semidefinite(R_kernel)
    pi = tensortree.model.backend.make_equilibrium(pi_kernel)
    B = tensortree.model.backend.make_branch_lengths(B_kernel)
    return R, pi, B

I0000 00:00:1733389433.858746  335797 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733389433.894151  335797 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733389433.894396  335797 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733389433.897379  335797 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [6]:
# check initial values
R, pi, B = make_R_pi_B(R_kernel, pi_kernel, B_kernel)
Q = tensortree.model.backend.make_rate_matrix(R, pi)
Q[0], pi, B

(<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[-1.        ,  0.33333334,  0.33333334,  0.33333334],
        [ 0.33333334, -1.        ,  0.33333334,  0.33333334],
        [ 0.33333334,  0.33333334, -1.        ,  0.33333334],
        [ 0.33333334,  0.33333334,  0.33333334, -1.        ]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.25, 0.25, 0.25, 0.25]], dtype=float32)>,
 <tf.Tensor: shape=(7, 1), dtype=float32, numpy=
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=float32)>)

In [7]:
%%time
# model training
lr = 0.05
num_steps = 100


@tf.function
def train_step(one_hot_leaves, leaf_names, tree):
    with tf.GradientTape() as tape:
        R, pi, B = make_R_pi_B(R_kernel, pi_kernel, B_kernel)
        Q = tensortree.model.backend.make_rate_matrix(R, pi)
        L = tensortree.model.loglik(one_hot_leaves, leaf_names, tree, Q, B, tf.math.log(pi), 
                            leaves_are_probabilities=True)
        loss = -tf.reduce_mean(L)
    grads = tape.gradient(loss, [R_kernel, pi_kernel, B_kernel])
    return loss, grads

optimizer = tf.keras.optimizers.Adam(learning_rate = lr)

for step in range(num_steps):
    loss, grads = train_step(one_hot_leaves, leaf_names, tree)
    dR, dpi, dB = grads

    optimizer.apply_gradients(zip([dR, dpi, dB], [R_kernel, pi_kernel, B_kernel]))

    # test output
    if step < 5 or step % 10 == 0:
        print(f"loss after epoch {step}={loss.numpy()}")

Cause: Unable to locate the source code of <function train_step at 0x700ce00fed40>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function train_step at 0x700ce00fed40>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.

2024-12-05 10:03:56.665052: I tensorflow/core/util/cuda_solvers.cc:178] Creating GpuSolver handles for stream 0x5a42d04a86c0


loss after epoch 0=6.580846309661865
loss after epoch 1=6.462332725524902
loss after epoch 2=6.357039451599121
loss after epoch 3=6.262740135192871
loss after epoch 4=6.179914951324463
loss after epoch 10=5.817017555236816
loss after epoch 20=5.374495029449463
loss after epoch 30=5.101579189300537
loss after epoch 40=4.952778339385986
loss after epoch 50=4.876732349395752
loss after epoch 60=4.847573280334473
loss after epoch 70=4.841607570648193
loss after epoch 80=4.838799476623535
loss after epoch 90=4.838010787963867
CPU times: user 3.83 s, sys: 775 ms, total: 4.6 s
Wall time: 4.38 s


In [8]:
R, pi, B = make_R_pi_B(R_kernel, pi_kernel, B_kernel)
Q = tensortree.model.backend.make_rate_matrix(R, pi)

In [9]:
pi, R[0], Q[0]

(<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.09933323, 0.19983937, 0.30054235, 0.4002851 ]], dtype=float32)>,
 <tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[0.       , 1.8742675, 0.8975281, 1.4307388],
        [1.8742675, 0.       , 1.0815687, 1.240525 ],
        [0.8975281, 1.0815687, 0.       , 1.6682485],
        [1.4307388, 1.240525 , 1.6682485, 0.       ]], dtype=float32)>,
 <tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[-1.2526371 ,  0.38552007,  0.2776439 ,  0.5894733 ],
        [ 0.19162866, -1.0373083 ,  0.33457553,  0.51110405],
        [ 0.09176498,  0.22246903, -1.0015628 ,  0.6873288 ],
        [ 0.14628145,  0.2551649 ,  0.51606077, -0.9175071 ]],
       dtype=float32)>)

In [10]:
true_pi = np.array([0.1, 0.2, 0.3, 0.4])

# normalized rate matrix
r = [10, 5, 8, 6, 7, 9]
mut = 2 * (true_pi[0] * true_pi[1] * r[0] + true_pi[0] * true_pi[2] * r[1] + true_pi[0] * true_pi[3] * r[2] + 
           true_pi[1] * true_pi[2] * r[3] + true_pi[1] * true_pi[3] * r[4] + true_pi[2] * true_pi[3] *r[5])
rnorm = r / mut
q = [rnorm[0] * true_pi[1], rnorm[1] * true_pi[2], rnorm[2] * true_pi[3], 
     rnorm[3] * true_pi[2], rnorm[4] * true_pi[3], rnorm[5] * true_pi[3]]
rnorm, q

(array([1.87265918, 0.93632959, 1.49812734, 1.12359551, 1.31086142,
        1.68539326]),
 [0.3745318352059925,
  0.28089887640449435,
  0.599250936329588,
  0.3370786516853932,
  0.5243445692883895,
  0.6741573033707864])

In [11]:
for learned_branch_length, true_branch_length in zip(B[:,0], tree.branch_lengths[:, 0]):
    print(learned_branch_length.numpy(), true_branch_length)

0.29544625 0.3017016
0.29963386 0.3013877
0.19719233 0.2005827
0.098468825 0.100148804
0.09867688 0.09924722
0.3048222 0.29687506
0.10516685 0.10143233
